In [1]:
# pacotes
import numpy as np
import pandas as pd

In [2]:
# oculta mensagens de avisos
import warnings
warnings.filterwarnings("ignore")

In [3]:
# pd.set_option('display.max_columns', None)

In [4]:
# pd.set_option('display.max_rows', None)

In [5]:
# criando um range de datas úteis
business_dates = pd.bdate_range('2005-01-01', '2019-12-31')

In [6]:
# criando um dataframe que guardará essas informações
df = pd.DataFrame(
    index=business_dates, 
    columns=['Number of Negative Market Cap Stocks',
            'Tickers']
    )

In [7]:
for bdate in business_dates:
    try:
        # formatando a data
        day = str(bdate)[:4] + str(bdate)[5:7] + str(bdate)[8:10]

        # tratando o dataframe de fatores
        factors_path = f'../../../input/factors/{day}.csv'
        factors = pd.read_csv(factors_path, index_col=0)
        factors = factors[factors['TAQ_TICKER'] != '<undefined>']

        # tickers com market cap negativo 
        negative_mktcap = factors[factors['MARKETCAP'] < 0]['TAQ_TICKER']

        # tornando o objeto uma lista
        list_negative_factors = list(negative_mktcap)

        """
        preenchendo os dados
        coluna = 0    'Number of Negative Market Cap Stocks'
        coluna = 1    'Tickers with Negative Market Cap'
        """
        df.loc[bdate][0] = len(list_negative_factors)
        df.loc[bdate][1] = list_negative_factors
    except:
        pass

In [8]:
df = df.dropna(axis=0)

In [9]:
df

,Number of Negative Market Cap Stocks,Tickers
2005-01-03,99,"[BFEN, UBMT, BKSC, PLE, STIZ, MHG, PFDC, PZA, ..."
2005-01-04,127,"[AMCE, NHTB, BFEN, AABC, UBMT, CEBK, ELMS, HOS..."
2005-01-05,117,"[AABC, CEBK, BKSC, CCDC, HOST, EBIX, MBLA, HAR..."
2005-01-06,144,"[AMCE, NHTB, BFEN, QUIP, AABC, BKSC, JEN, CFNB..."
2005-01-07,142,"[DGSE, AMCE, ESBK, DMC, PATR, BFEN, QUIP, UBMT..."
...,...,...
2019-12-24,70,"[AIRT, OFED, BOTJ, SFBC, GYRO, HVBC, ICCH, KBL..."
2019-12-26,55,"[STND, SFBC, ICCH, KBLM, BPRN, NAOV, BRPA, LAC..."
2019-12-27,56,"[OFED, IROQ, AAMC, PIH, CNFR, GYRO, KBLM, WRLS..."
2019-12-30,48,"[HFBL, GYRO, ICCH, KBLM, NAOV, BRPA, MUDS, TIB..."


In [10]:
df.to_csv('../../../output/data/negative_mktvalue.csv', sep=',', encoding='utf-8')

Vamos analisar individualmente esses ticker que tiveram Market Cap negativo em pelo menos um dia.

In [11]:
all_tickers_negative_mktcap = []

for date in df.index:
    for i in range(len(df['Tickers'].loc[date])):
        all_tickers_negative_mktcap.append(df['Tickers'].loc[date][i])

In [12]:
# verificando o tamanho
len(all_tickers_negative_mktcap)

312963

In [13]:
# uma lista tem a função unique(), transformamos em um set (só ficam os elementos uma vez) e transformamos de volta em uma lista
all_tickers_negative_mktcap_unique = list(set(all_tickers_negative_mktcap))

In [14]:
# verificando o tamanho
len(all_tickers_negative_mktcap_unique)

2868

In [15]:
day = '20050103'
# tratando o dataframe de fatores
factors_path = f'../../../input/factors/{day}.csv'
factors = pd.read_csv(factors_path, index_col=0)
factors = factors[factors['TAQ_TICKER'] != '<undefined>']

In [16]:
permnos = []

for ticker in all_tickers_negative_mktcap_unique:
    try:
        permnos.append(factors[factors['TAQ_TICKER'] == ticker].index[0])
    except:
        pass

In [17]:
# dataframe que vai guardar todos as series das ações que tiveram market cap negativo
df_ = pd.DataFrame(index=business_dates)

In [18]:
for permno in permnos:
    try:
        # pegando dados para cada permno
        path = f'../../../../factors_database_/output/by_permno/{permno}.csv'
        temp = pd.read_csv(path, index_col=0)

        # nomearemos a coluna com o nome da ação
        col_name = temp['TAQ_TICKER'][0]
        # precisamos deixar os índices como datetime
        temp.index = pd.to_datetime(temp.index)

        # adicionando coluna ao dataframe
        df_[col_name] = temp['MARKETCAP']
    except:
        pass

In [19]:
# dropando linhas cujo todas colunas são NaN
df_ = df_.dropna(axis=0, how='all')

In [20]:
df_

,PHLY,COOP,GBTS,BWCF,SYNL,MRVC,AMPH,NLCI,CAP,OSKY,...,SMXC,ENCY,MBLA,AMBK,DEVC,WEFN,PSTI,PMID,CFS,PBCP
2005-01-03,1466965.64,77819.20,106876.950,-97730.145,59537.80,378825.72,26899.650,56396.5,13076.86,74832.450,...,29647.600,572809.02,39110.960,60853.775,88204.71,21835.000,473520.32,18166.320,46729.20,602352.32
2005-01-04,1477657.16,76674.80,105878.100,97021.800,60904.34,359051.85,27055.590,56247.5,12014.78,73459.584,...,29729.521,572809.02,39140.500,61034.350,86624.40,21572.980,465064.60,18376.944,46061.64,603270.54
2005-01-05,1454492.20,-77218.39,106217.709,98739.000,59116.40,337196.52,26275.890,56620.0,11384.17,73969.720,...,30193.740,571650.66,-38741.710,60962.120,88146.18,-21703.990,450569.08,18315.512,45894.75,606484.31
2005-01-06,1463179.06,76789.24,106870.291,-96656.895,59116.40,347603.82,27133.560,58110.0,12346.68,73219.520,...,31208.000,590763.60,38921.904,60384.280,87795.00,20979.068,463554.65,17990.800,45060.30,604647.87
2005-01-07,1464070.02,78648.89,106610.590,94660.650,-60049.50,343440.90,-26782.695,57737.5,12413.06,73294.540,...,30778.890,595976.22,38047.520,61034.350,87970.59,20786.920,451475.05,17903.040,44893.41,601893.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,NaN,NaN,NaN,NaN,115093.16,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4273272.90
2019-12-26,NaN,NaN,NaN,NaN,117803.36,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4275293.36
2019-12-27,NaN,NaN,NaN,NaN,119519.82,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4238925.08
2019-12-30,NaN,NaN,NaN,NaN,118435.74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4242966.00


In [21]:
df_.to_csv('../../../output/data/marketcap.csv', sep=',', encoding='utf-8')